### Apache Iceberg + Trino Performance Evaluation

This notebook compares the performance of Apache Iceberg with Trino against the current Parquet implementation for handling MultiQC data.

In [2]:
import time
from datetime import datetime

import os
from pathlib import Path
from cloudpathlib import AnyPath
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import trino
from trino.auth import BasicAuthentication

import pyarrow as pa
from pyiceberg.catalog.rest import RestCatalog
from cloudpathlib import S3Path
from dotenv import load_dotenv

In [3]:
# Configuration parameters
NUM_RUNS = 10
NUM_MODULES = 10
NUM_SECTIONS_PER_MODULE = 5
NUM_SAMPLES_PER_MODULE = 10
NUM_METRICS_PER_MODULE = 20

# Parquet setup
dir_name = f"data/{NUM_RUNS}runs_{NUM_MODULES}mod_{NUM_SAMPLES_PER_MODULE}samples_{NUM_METRICS_PER_MODULE}metrics"
parquet_path = Path(dir_name) / "parquet"

# Trino connection parameters
TRINO_HOST = "trino-coordinator"
TRINO_PORT = 8080
TRINO_USER = "trino"
TRINO_CATALOG = "iceberg"
TRINO_SCHEMA = "default"

load_dotenv()
# Define catalog connection details (replace variables)
TOKEN = "M6VUa1BsUg4FjOtGd-vdj_DDmkySzLOACFvWY7JH"
WAREHOUSE = "8234d07c9c28a6f6c380fe45731ba8e4_megaqc-test"
CATALOG_URI = "https://catalog.cloudflarestorage.com/8234d07c9c28a6f6c380fe45731ba8e4/megaqc-test"

In [ ]:
"""
PREFIX="simulated/100runs_10mod_100samples_20metrics/parquet"
BUCKET="megaqc-test"
find ./10runs_10mod_10samples_20metrics -type f -print0 |
  while IFS= read -r -d '' file; do
    key="$PREFIX/$(basename "$file")"
    npx wrangler r2 object put "$BUCKET/$key" --file "$file"
  done
"""

In [4]:
# Connect to R2 Data Catalog
catalog = RestCatalog(
    name="multiqc-small",
    token=TOKEN,
    warehouse=WAREHOUSE,
    uri=CATALOG_URI,
)

# Create default namespace
catalog.create_namespace("small")

In [13]:
parquet_path

PosixPath('data/10runs_10mod_10samples_20metrics/parquet')

In [14]:
# Load from parquet with pyarrow
# Read partitioned parquet dataset using pyarrow, partitioned by run_id
dataset = ds.dataset(
    str(parquet_path),
    format="parquet",
    partitioning=ds.partitioning(
        pa.schema([("run_id", pa.string())])
    )
)

pa_table = dataset.to_table()

In [6]:
catalog.list_tables("default")

[('default', 'multiqc_metrics')]

In [15]:
iceberg_table = catalog.create_table(
    ("small", "multiqc_metrics_partitioned"),
    schema=pa_table.schema,
)
iceberg_table = catalog.load_table(("small", "multiqc_metrics_partitioned"))
iceberg_table.append(pa_table)

In [16]:
import polars as pl
iceberg_table.to_polars().filter(pl.col("metric_name") == "metric_0").collect()

timestamp,module_id,module_name,module_url,module_comment,module_anchor,module_doi,sample_id,metric_name,val_raw,val_raw_type,val_mod,val_mod_type,val_fmt,metric_min,metric_max,metric_dmin,metric_dmax,metric_scale,metric_color,metric_type,metric_namespace,metric_placement,metric_shared_key,entity_type,section_id,section_name,section_anchor,section_description,section_module,section_module_anchor,section_module_info,section_comment,section_helptext,section_content_before_plot,section_content,section_plot,section_print_section,section_plot_anchor,section_ai_summary,run_id
str,str,str,str,str,str,str,str,str,f64,str,f64,str,str,f64,f64,f64,f64,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,str,str
"""2025-04-25T18:28:44.697160""","""module_0""","""Module 0""","""http://example.com/module/0""","""This is module 0""","""anchor_0""","""QOHJQdizjKjUFrlYkgbw""","""roGIXDTvEr""","""metric_0""",43.861322,"""str""",43.861322,"""float""","""43.86""",0.666426,99.714283,2.819649,9.158384,"""Paired""","""#4ee65e""","""percentage""","""AfJCIBqehn""",874.0,"""read_count""","""sample_metric""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""run_id=run_0"""
"""2025-04-25T18:28:44.697160""","""module_0""","""Module 0""","""http://example.com/module/0""","""This is module 0""","""anchor_0""","""QOHJQdizjKjUFrlYkgbw""","""eiTOlKslUD""","""metric_0""",43.131013,"""str""",43.131013,"""str""","""43.13""",0.666426,99.714283,2.819649,9.158384,"""Paired""","""#4ee65e""","""percentage""","""AfJCIBqehn""",874.0,"""read_count""","""sample_metric""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""run_id=run_0"""
"""2025-04-25T18:28:44.697160""","""module_0""","""Module 0""","""http://example.com/module/0""","""This is module 0""","""anchor_0""","""QOHJQdizjKjUFrlYkgbw""","""aKlGxHIByU""","""metric_0""",8.21306,"""str""",8.21306,"""int""","""8.21""",0.666426,99.714283,2.819649,9.158384,"""Paired""","""#4ee65e""","""percentage""","""AfJCIBqehn""",874.0,"""read_count""","""sample_metric""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""run_id=run_0"""
"""2025-04-25T18:28:44.697160""","""module_0""","""Module 0""","""http://example.com/module/0""","""This is module 0""","""anchor_0""","""QOHJQdizjKjUFrlYkgbw""","""QnHSLjEpJo""","""metric_0""",12.053803,"""float""",12.053803,"""float""","""12.05""",0.666426,99.714283,2.819649,9.158384,"""Paired""","""#4ee65e""","""percentage""","""AfJCIBqehn""",874.0,"""read_count""","""sample_metric""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""run_id=run_0"""
"""2025-04-25T18:28:44.697160""","""module_0""","""Module 0""","""http://example.com/module/0""","""This is module 0""","""anchor_0""","""QOHJQdizjKjUFrlYkgbw""","""tWpuhVUMdV""","""metric_0""",81.437652,"""int""",81.437652,"""int""","""81.44""",0.666426,99.714283,2.819649,9.158384,"""Paired""","""#4ee65e""","""percentage""","""AfJCIBqehn""",874.0,"""read_count""","""sample_metric""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""run_id=run_0"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025-04-25T18:28:44.744532""","""module_9""","""Module 9""","""http://example.com/module/9""","""This is module 9""","""anchor_9""","""zpzigLMURbPatxCKZlgI""","""AFoMMiSYYJ""","""metric_0""",15.473939,"""float""",15.473939,"""float""","""15.47""",8.912544,94.201981,1.008753,9.146479,"""Pastel1""","""#62bca3""","""percentage""","""lKmyYQsyqi""",990.0,"""base_count""","""sample_metric""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""run_id=run_9"""
"""2025-04-25T18:28:44.744532""","""module_9""","""Module 9""","""http://example.com/module/9""","""This is module 9""","""anchor_9""","""zpzigLMURbPatxCKZlgI""","""NySndZFRXU""","""metric_0""",35.131007,"""int""",35.131007,"""str""","""35.13""",8.912544,94.201981,1.008753,9.146479,"

#### Querying with Trino

In [ ]:
conn = trino.dbapi.connect(
    host="localhost",          # or EC2 / k8s hostname
    port=8080,
    user="vlad",
    catalog="r2",              # <- matches r2.properties
    schema="default",
)
cursor = conn.cursor()
cursor.execute("SELECT count(*) FROM multiqc_metrics_partitioned")
cursor.execute("""
CREATE TABLE IF NOT EXISTS metrics (
    run_id VARCHAR,
    module_id VARCHAR,
    metric_name VARCHAR,
    value DOUBLE
)
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['run_id']
)
""")

TrinoConnectionError: failed to execute: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /v1/statement (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1543a8f30>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [5]:
def query_single_metric_parquet(parquet_dir, metric_name="metric_0"):
    """Query Parquet files to retrieve specific metric values using PyArrow"""
    print(f"Querying Parquet files for metrics with name: {metric_name}")
    start_time = time.time()

    # Read the Parquet files with partitioning information
    dataset = ds.dataset(parquet_dir, format="parquet", partitioning="hive")

    # Define filter condition for the metric name
    filter_expr = (ds.field("metric_name") == metric_name)
    # Read the filtered data
    table = dataset.to_table(filter=filter_expr)
    # Convert to pandas DataFrame if needed
    df = table.to_pandas()

    end_time = time.time()
    elapsed = end_time - start_time
    
    print(f"Query found {len(df)} records in {elapsed:.4f} seconds")
    return df, elapsed


def query_single_module_parquet(parquet_dir, run_id="run_0", module_id="module_0"):
    """Query Parquet files to retrieve specific module data using PyArrow"""
    print(f"Querying Parquet files for run_id={run_id} and module_id={module_id}")
    start_time = time.time()

    # Read the Parquet files with partitioning information
    dataset = ds.dataset(parquet_dir, format="parquet", partitioning="hive")

    # Define filter condition for the run_id and module_id
    filter_expr = (ds.field("run_id") == run_id) & (ds.field("module_id") == module_id)
    # Read the filtered data
    table = dataset.to_table(filter=filter_expr)
    # Convert to pandas DataFrame if needed
    df = table.to_pandas()
    
    end_time = time.time()
    elapsed = end_time - start_time
    
    print(f"Query found {len(df)} records in {elapsed:.4f} seconds")
    return df, elapsed


def run_parquet_benchmark(parquet_dir, num_runs=10, num_modules=10, 
                         num_samples_per_module=100, num_metrics_per_module=20):
    """Run a complete Parquet benchmark"""
    print("-" * 80)
    print("PARQUET BENCHMARK")
    print("-" * 80)
    
    print("\nGenerating sample data with:")
    print(f"- {num_runs} runs")
    print(f"- {num_modules} modules per run")
    print(f"- {num_samples_per_module} samples per module")
    print(f"- {num_metrics_per_module} metrics per module")
    
    # Storage benchmark
    current_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    benchmark_dir = f"{parquet_dir}/{current_time}"
    
    # Query benchmarks
    print("\nRunning query benchmarks:")
    
    # Query by metric name
    _, metric_query_time = query_single_metric_parquet(benchmark_dir)
    
    # Query by run_id and module_id
    _, module_query_time = query_single_module_parquet(benchmark_dir)
    
    # Summary
    print("\nPARQUET BENCHMARK SUMMARY:")
    print(f"Query by metric time: {metric_query_time:.4f} seconds")
    print(f"Query by module time: {module_query_time:.4f} seconds")
    
    return {
        "metric_query_time": metric_query_time,
        "module_query_time": module_query_time
    }


# MinIO/S3 configuration
PARQUET_BUCKET = "s3://megaqc-test/parquet_data"

# Set MinIO credentials for local testing
# If using AWS S3 directly, these would be your AWS credentials
# os.environ["AWS_ACCESS_KEY_ID"] = "minio"
# os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
# os.environ["AWS_ENDPOINT_URL"] = "http://minio:9000"
# os.environ["AWS_REGION"] = "us-east-1"

# Run the benchmark with smaller dataset for testing
results = run_parquet_benchmark(
    PARQUET_BUCKET,
    num_runs=5,
    num_modules=5,
    num_samples_per_module=10,
    num_metrics_per_module=5
)

--------------------------------------------------------------------------------
PARQUET BENCHMARK
--------------------------------------------------------------------------------

Generating sample data with:
- 5 runs
- 5 modules per run
- 10 samples per module
- 5 metrics per module
Flattening data...
Creating DataFrame...
Creating Parquet Table...
Writing to Parquet file s3://megaqc-test/parquet_data/2025-04-23-13-46-14...
Parquet storage time: 7.2234 seconds

Running query benchmarks:
Querying Parquet files for metrics with name: metric_0
Query found 250 records in 1.2624 seconds
Querying Parquet files for run_id=run_0 and module_id=module_0
Query found 50 records in 1.2209 seconds

PARQUET BENCHMARK SUMMARY:
Storage time: 7.2234 seconds
Query by metric time: 1.2624 seconds
Query by module time: 1.2209 seconds


In [32]:
from pyiceberg.catalog import load_catalog

cat = load_catalog(
    "multiqc",
    type="sql",
    uri="sqlite:////Users/vlad/multiqc_iceberg.db",
    warehouse="s3://megaqc-test/iceberg_data/2025-04-23-13-46-14",
)

arrow_tbl = pa.Table.from_pandas(df, preserve_index=False)

cat.create_namespace("multiqc")
cat.create_table(
    identifier="multiqc.metrics",
    schema=arrow_tbl.schema,
)

# Load the table and append the new snapshot
table = cat.load_table("multiqc.metrics")
table.append(arrow_tbl)  # fast-append commit

In [6]:
# Benchmark script for Iceberg storage and querying with Trino

def create_trino_connection(
        host="trino-coordinator", port=8080, 
        user="trino", catalog="iceberg", schema="default"
    ):
    """Create a connection to Trino"""
    try:
        conn = trino.dbapi.connect(
            host=host,
            port=port,
            user=user,
            catalog=catalog,
            schema=schema,
        )
        print("Connected to Trino successfully!")
        return conn
    except Exception as e:
        print(f"Error connecting to Trino: {e}")
        return None


def query_single_metric_iceberg(conn, metric_name="metric_0"):
    """Query Iceberg table to retrieve specific metric values using Trino"""
    print(f"Querying Iceberg table for metrics with name: {metric_name}")
    start_time = time.time()

    # Execute query through Trino
    cursor = conn.cursor()
    query = f"""
        SELECT * FROM metrics
        WHERE metric_name = '{metric_name}'
    """
    
    try:
        cursor.execute(query)
        # Fetch all results
        results = cursor.fetchall()
        
        # Convert to DataFrame
        columns = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(results, columns=columns)
        
        end_time = time.time()
        elapsed = end_time - start_time
        
        print(f"Query found {len(df)} records in {elapsed:.4f} seconds")
        return df, elapsed
    
    except Exception as e:
        print(f"Error querying data: {e}")
        return pd.DataFrame(), -1


def query_single_module_iceberg(conn, run_id="run_0", module_id="module_0"):
    """Query Iceberg table to retrieve specific module data using Trino"""
    print(f"Querying Iceberg table for run_id={run_id} and module_id={module_id}")
    start_time = time.time()

    # Execute query through Trino
    cursor = conn.cursor()
    query = f"""
        SELECT * FROM metrics
        WHERE run_id = '{run_id}' AND module_id = '{module_id}'
    """
    
    try:
        cursor.execute(query)
        # Fetch all results
        results = cursor.fetchall()
        
        # Convert to DataFrame
        columns = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(results, columns=columns)
        
        end_time = time.time()
        elapsed = end_time - start_time
        
        print(f"Query found {len(df)} records in {elapsed:.4f} seconds")
        return df, elapsed
    
    except Exception as e:
        print(f"Error querying data: {e}")
        return pd.DataFrame(), -1


def run_iceberg_benchmark(num_runs=10, num_modules=10, 
                         num_samples_per_module=100, num_metrics_per_module=20):
    """Run a complete Iceberg benchmark"""
    print("-" * 80)
    print("ICEBERG BENCHMARK")
    print("-" * 80)
    
    print("\nGenerating sample data with:")
    print(f"- {num_runs} runs")
    print(f"- {num_modules} modules per run")
    print(f"- {num_samples_per_module} samples per module")
    print(f"- {num_metrics_per_module} metrics per module")
    
    # Generate test data
    data = generate_all_data(
        num_runs, num_modules, num_samples_per_module, num_metrics_per_module
    )
    
    # Connect to Trino
    conn = create_trino_connection()
    if not conn:
        print("Failed to connect to Trino. Aborting Iceberg benchmark.")
        return {
            "storage_time": -1,
            "metric_query_time": -1,
            "module_query_time": -1
        }
    
    # Initialize Iceberg schema
    if not init_iceberg_schema(conn):
        print("Failed to initialize Iceberg schema. Aborting Iceberg benchmark.")
        return {
            "storage_time": -1,
            "metric_query_time": -1,
            "module_query_time": -1
        }
    
    # Clear existing data
    try:
        cursor = conn.cursor()
        cursor.execute("DELETE FROM metrics")
        print("Cleared existing data from metrics table")
    except Exception as e:
        print(f"Error clearing metrics table: {e}")
    
    # Storage benchmark
    storage_time = store_in_iceberg(data, conn)
    
    # Query benchmarks
    print("\nRunning query benchmarks:")
    
    # Query by metric name
    _, metric_query_time = query_single_metric_iceberg(conn)
    
    # Query by run_id and module_id
    _, module_query_time = query_single_module_iceberg(conn)
    
    # Summary
    print("\nICEBERG BENCHMARK SUMMARY:")
    print(f"Storage time: {storage_time:.4f} seconds")
    print(f"Query by metric time: {metric_query_time:.4f} seconds")
    print(f"Query by module time: {module_query_time:.4f} seconds")
    
    return {
        "storage_time": storage_time,
        "metric_query_time": metric_query_time,
        "module_query_time": module_query_time
    }


# Run the benchmark with smaller dataset for testing
results = run_iceberg_benchmark(
    num_runs=5,
    num_modules=5,
    num_samples_per_module=10,
    num_metrics_per_module=5
) 

--------------------------------------------------------------------------------
ICEBERG BENCHMARK
--------------------------------------------------------------------------------

Generating sample data with:
- 5 runs
- 5 modules per run
- 10 samples per module
- 5 metrics per module
Connected to Trino successfully!
Error creating Iceberg table: failed to execute: HTTPConnectionPool(host='trino-coordinator', port=8080): Max retries exceeded with url: /v1/statement (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x113ee9760>: Failed to resolve 'trino-coordinator' ([Errno 8] nodename nor servname provided, or not known)"))
Failed to initialize Iceberg schema. Aborting Iceberg benchmark.


In [9]:
conn = trino.dbapi.connect(
    host="localhost",
    port=8080,
    user="trino",
    catalog="iceberg",
    schema="default"
)
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS metrics (
    run_id VARCHAR,
    module_id VARCHAR,
    metric_name VARCHAR,
    value DOUBLE
)
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['run_id']
)
""")

TrinoConnectionError: failed to execute: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /v1/statement (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1081c19d0>: Failed to establish a new connection: [Errno 61] Connection refused'))